## Creating Enumerations

A new enumeration is defined using the class syntax by subclassing Enum and adding class attributes describing the values

In [1]:
import enum

In [21]:
class BugStatus(enum.Enum):
    new = 7
    incomplete = 6
    invalid = 5
    wont_fix = 4
    in_progress = 3
    fix_committed = 2
    fix_released = 1

In [3]:
print("member name: {}".format(BugStatus.wont_fix.name))
print("member value: {}".format(BugStatus.wont_fix.value))

member name: wont_fix
member value: 4


## Access to enumeration members and their attributes

In [22]:
BugStatus(7)

<BugStatus.new: 7>

In [23]:
# if you want access enum members by name, use item access

In [ ]:
BugStatus['']

##  Iteration

Iterating over the enum class pruduces the indivisual members of the member

In [4]:
for status in BugStatus:
    print("{}:{}".format(status.name, status.value))

new:7
incomplete:6
invalid:5
wont_fix:4
in_progress:3
fix_committed:2
fix_released:1


**Note**:
The members are produced in the order they are declared in the class definition. The names and values are not used to sort them.

## Comparing Enums

Because enumeration members are not ordered. They only support equality and identify test

In [5]:
actual_state = BugStatus.wont_fix
desired_state = BugStatus.fix_released

print('Equality:',
      actual_state == desired_state,
      actual_state == BugStatus.wont_fix)
print('Identity:',
      actual_state is desired_state,
      actual_state is BugStatus.wont_fix)
print('Ordered by value:')
try:
    print('\n'.join('  ' + s.name for s in sorted(BugStatus)))
except TypeError as err:
    print('  Cannot sort: {}'.format(err))

Equality: False True
Identity: False True
Ordered by value:
  Cannot sort: '<' not supported between instances of 'BugStatus' and 'BugStatus'


### IntEnum

use the `IntEnum` class for enumrations where the members need to behave more like numbers, for example, to support comparisons

In [7]:
class BugStatusInt(enum.IntEnum):

    new = 7
    incomplete = 6
    invalid = 5
    wont_fix = 4
    in_progress = 3
    fix_committed = 2
    fix_released = 1

In [9]:
actual_state = BugStatusInt.wont_fix
desired_state = BugStatusInt.fix_released

print('Equality:',
      actual_state == desired_state,
      actual_state == BugStatusInt.wont_fix)
print('Identity:',
      actual_state is desired_state,
      actual_state is BugStatusInt.wont_fix)

print("comparison: ")
print("new is bigger then invalid:", BugStatusInt.new > BugStatusInt.invalid)

print('Ordered by value:')
print('\n'.join('  ' + s.name for s in sorted(BugStatusInt)))


Equality: False True
Identity: False True
comparison: 
new is bigger then invalid: True
Ordered by value:
  fix_released
  fix_committed
  in_progress
  wont_fix
  invalid
  incomplete
  new


## Unique Enumeration Values

Enum members with the same value are tracked as alias references to the same member object. Aliases do not cause repeated values to be present in the iterator for the Enum

In [12]:
class BugStatusUnique(enum.Enum):

    new = 7
    incomplete = 6
    invalid = 5
    wont_fix = 4
    in_progress = 3
    fix_committed = 2
    fix_released = 1

    by_design = 4
    closed = 1


for status in BugStatusUnique:
    print('{:15} = {}'.format(status.name, status.value))

print('\nSame: by_design is wont_fix: ',
      BugStatusUnique.by_design is BugStatusUnique.wont_fix)
print('Same: closed is fix_released: ',
      BugStatusUnique.closed is BugStatusUnique.fix_released)

new             = 7
incomplete      = 6
invalid         = 5
wont_fix        = 4
in_progress     = 3
fix_committed   = 2
fix_released    = 1

Same: by_design is wont_fix:  True
Same: closed is fix_released:  True


**To require all members to have unique values, add the decorator**

In [13]:
@enum.unique
class BugStatusUniqueDecorator(enum.Enum):

    new = 7
    incomplete = 6
    invalid = 5
    wont_fix = 4
    in_progress = 3
    fix_committed = 2
    fix_released = 1

    # This will trigger an error with unique applied.
    by_design = 4
    closed = 1

ValueError: duplicate values found in <enum 'BugStatusUniqueDecorator'>: by_design -> wont_fix, closed -> fix_released

Members with repeated values trigger a `ValueError` exception when the `Enum` class being interpreted

## Create Enumerations Programmatically

In [14]:
BugStatus = enum.Enum(
    value='BugStatus',
    names=('fix_released fix_committed in_progress '
           'wont_fix invalid incomplete new'),
)

print('Member: {}'.format(BugStatus.new))

print('\nAll members:')
for status in BugStatus:
    print('{:15} = {}'.format(status.name, status.value))

Member: BugStatus.new

All members:
fix_released    = 1
fix_committed   = 2
in_progress     = 3
wont_fix        = 4
invalid         = 5
incomplete      = 6
new             = 7


value: the name of the enumration
names: lists the member of the enumeration, if a string is passed. it is split on the whitespace and commas. the value to names is starting with 1

If you want control the value associated with members, the names string can be replaced with a sequence of 2 part tuples or a dictionary mapping names to values

In [15]:

BugStatus = enum.Enum(
    value='BugStatus',
    names=[
        ('new', 7),
        ('incomplete', 6),
        ('invalid', 5),
        ('wont_fix', 4),
        ('in_progress', 3),
        ('fix_committed', 2),
        ('fix_released', 1),
    ],
)

print('All members:')
for status in BugStatus:
    print('{:15} = {}'.format(status.name, status.value))

All members:
new             = 7
incomplete      = 6
invalid         = 5
wont_fix        = 4
in_progress     = 3
fix_committed   = 2
fix_released    = 1


## Non-integer Member Values

Enum member values are not restricted to integers. In fact, any type of object can be associated with a member. If the value is a tuple, the members are passed as individual arguments to __init__().

In [16]:

class BugStatus(enum.Enum):

    new = (7, ['incomplete',
               'invalid',
               'wont_fix',
               'in_progress'])
    incomplete = (6, ['new', 'wont_fix'])
    invalid = (5, ['new'])
    wont_fix = (4, ['new'])
    in_progress = (3, ['new', 'fix_committed'])
    fix_committed = (2, ['in_progress', 'fix_released'])
    fix_released = (1, ['new'])

    def __init__(self, num, transitions):
        self.num = num
        self.transitions = transitions

    def can_transition(self, new_state):
        return new_state.name in self.transitions


print('Name:', BugStatus.in_progress)
print('Value:', BugStatus.in_progress.value)
print('Custom attribute:', BugStatus.in_progress.transitions)
print('Using attribute:',
      BugStatus.in_progress.can_transition(BugStatus.new))

Name: BugStatus.in_progress
Value: (3, ['new', 'fix_committed'])
Custom attribute: ['new', 'fix_committed']
Using attribute: True


In [20]:
BugStatus.new.transitions

['incomplete', 'invalid', 'wont_fix', 'in_progress']